In [1]:
# import all depedencies first cell
import numpy as np
import pandas as pd
from sklearn import linear_model, metrics, cross_validation

In [2]:
# define functions in one cell, call when you need
def read_csv(path):
    return pd.read_csv(path)

In [3]:
path = '../../assets/datasets/bikeshare.csv'
bikeshare = read_csv(path)

In [5]:
# get the data dictionary of bikeshare to see possible feature candidates
bikeshare.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
instant       17379 non-null int64
dteday        17379 non-null object
season        17379 non-null int64
yr            17379 non-null int64
mnth          17379 non-null int64
hr            17379 non-null int64
holiday       17379 non-null int64
weekday       17379 non-null int64
workingday    17379 non-null int64
weathersit    17379 non-null int64
temp          17379 non-null float64
atemp         17379 non-null float64
hum           17379 non-null float64
windspeed     17379 non-null float64
casual        17379 non-null int64
registered    17379 non-null int64
cnt           17379 non-null int64
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


In [6]:
bikeshare.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [7]:
# assume we want 2 features, 'temp' and 'hum' and weathersit, because
# weathersit is categorical we must make dummy variables
# create a new dataframe with only features temp and hum, 
# and all dummy variables required for weathersit (hint n - 1 classes)
# hint (pd.get_dummies(df_column)) will transform your variable 
# into n dummy classes
# hint, use .join to merge two dataframes on a common key (default inner join on index)
# call your features dataframe 'modeldata' and your y response column 'y'

modeldata = bikeshare[["temp","hum","weathersit"]]
y = bikeshare["casual"]
modeldata = modeldata.join(pd.get_dummies(modeldata["weathersit"],prefix="weathersit"))
del modeldata["weathersit"]
print modeldata.groupby("weathersit_1",as_index="False").count()
print modeldata.groupby("weathersit_2",as_index="False").count()
print modeldata.groupby("weathersit_3",as_index="False").count()
print modeldata.groupby("weathersit_4",as_index="False").count()
del modeldata["weathersit_1"]

               temp    hum  weathersit_2  weathersit_3  weathersit_4
weathersit_1                                                        
0.0            5966   5966          5966          5966          5966
1.0           11413  11413         11413         11413         11413
               temp    hum  weathersit_1  weathersit_3  weathersit_4
weathersit_2                                                        
0.0           12835  12835         12835         12835         12835
1.0            4544   4544          4544          4544          4544
               temp    hum  weathersit_1  weathersit_2  weathersit_4
weathersit_3                                                        
0.0           15960  15960         15960         15960         15960
1.0            1419   1419          1419          1419          1419
               temp    hum  weathersit_1  weathersit_2  weathersit_3
weathersit_4                                                        
0.0           17376  17376        

In [8]:
# kf = cross_validation.KFold(len(modeldata), n_folds=5, shuffle=True)

# you must define modeldata in cell above for kf assignment to run
# kf returns a dictionary composed of key-value pairs

logiReg = linear_model.LinearRegression()
model = logiReg.fit(modeldata,y)

kf = cross_validation.KFold(len(modeldata), n_folds=5, shuffle=True)

In [9]:
lm_obj = linear_model.LinearRegression() # instaniate only one time
scores = [] # we will append mse scores from each iteration in kf
for train_index, test_index in kf: # for (key,value) in dictionary
    x_train = modeldata.iloc[train_index] # get new set each iteration
    y_train = y.iloc[train_index]
    
    x_test = modeldata.iloc[test_index] # get new test data each iteration
    y_test = y.iloc[test_index]
        
    lm = lm_obj.fit(x_train, y_train) # fit new model each iteration
    x_test_pred = lm.predict(x_test)

    mse = metrics.mean_squared_error(y_test,x_test_pred) 
    # get new mse each iteration
    scores.append(mse) # append mse scores from each model to scores list

In [10]:
# print the mean mse score from all iterations, explain output
print scores

[1542.6000055299141, 1729.4198304373942, 1561.4289459160243, 1812.6976367001416, 1721.5799408732107]


In [22]:
# fit a regression model on all the model and outcome data (modeldata and y)
# hint use, linear_model.LinearRegression().fit()
# get predictions from fitted model using same model data
# calculate MSE and interpret 
# hint MSE = SSE/n = (y - y_est)/n, y_est = lm.predict(modeldata)

logiReg = linear_model.LinearRegression()
model = logiReg.fit(modeldata,y)
predictions = model.predict(modeldata)

print predictions

 [ -9.84732045 -11.26098864 -11.26098864 ...,  10.04881544  13.40926411
   5.8482546 ]


In [21]:
print "Accuracy score: ", metrics.accuracy_score(y.values,predictions)
print "Precision score: ", metrics.precision_score(y,predictions)
print "Recall score: ", metrics.recall_score(y,predictions)

Accuracy score: 

ValueError: Can't handle mix of multiclass and continuous

In [13]:
# fit a lasso regression model on all the model and outcome data (modeldata and y)
# hint use, linear_model.Lasso().fit()
# get predictions from fitted model using same model data
# calculate MSE and interpret
# hint MSE = SSE/n = (y - y_est)/n, y_est = lm.predict(modeldata)

logiReg = linear_model.LogisticRegression()
logiReg.penalty = 'l1'
model = logiReg.fit(modeldata,y)
predictions = model.predict(modeldata)

print predictions

[0 0 0 ..., 0 0 0]


In [14]:
print "Accuracy score: ", metrics.accuracy_score(y.values,predictions)
print "Precision score: ", metrics.precision_score(y,predictions)
print "Recall score: ", metrics.recall_score(y,predictions)

Accuracy score:  0.0924679210541
Precision score:  0.0185296344875
Recall score:  0.0924679210541


C:\Users\Alex Tam\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1203: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
C:\Users\Alex Tam\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1304: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sampl

In [15]:
# fit a Ridge regression model on all the model and outcome data (modeldata and y)
# hint use, linear_model.Ridge().fit()
# get predictions from fitted model using same model data
# calculate MSE and interpret
# hint MSE = SSE/n = (y - y_est)/n, y_est = lm.predict(modeldata)

In [16]:
logiReg = linear_model.LogisiticRegression()
logiReg.penalty = 'l2'
model = logiReg.fit(modeldata,y)
predictions = model.predict(modeldata)


AttributeError: 'module' object has no attribute 'LogisiticRegression'

In [ ]:
print "Accuracy score: ", metrics.accuracy_score(y.values,predictions)
print "Precision score: ", metrics.precision_score(y,predictions)
print "Recall score: ", metrics.recall_score(y,predictions)